In [1]:
import glob
import pandas as pd
import os
import json

import warnings; warnings.filterwarnings('ignore')

/Users/gmr/.pyenv/versions/3.9.7/envs/lsde/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Daily Data

In [65]:
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/nice.parquet" "../../data/website/databricks/hourly/sep2019.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/sep2018.parquet" "../../data/website/databricks/hourly/sep2018.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/oct2019.parquet" "../../data/website/databricks/hourly/oct2019.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedata123/aug2019.parquet" "../../data/website/databricks/hourly/aug2019.parquet"

In [12]:
df = pd.read_parquet("../../data/website/databricks/hourly/aug2019.parquet").drop_duplicates()
df["date"] = pd.to_datetime(df["timestamp"], unit="s")
df["year"] = df.date.dt.year
df["month"] = df.date.dt.month
df["day"] = df.date.dt.day
df["hour"] = df.date.dt.hour
df

,timestamp,domain,trafficType,accessType,y,x,date,year,month,day,hour
0,1566896400,ru.wikipedia,spider,desktop,4,40,2019-08-27 09:00:00,2019,8,27,9
1,1566889200,ru.wikipedia,spider,mobile-web,1,3241,2019-08-27 07:00:00,2019,8,27,7
2,1566882000,en.wikipedia,spider,desktop,10,61,2019-08-27 05:00:00,2019,8,27,5
3,1566871200,en.wikipedia,spider,desktop,14,14,2019-08-27 02:00:00,2019,8,27,2
4,1566864000,en.wikipedia,user,desktop,3,330,2019-08-27 00:00:00,2019,8,27,0
...,...,...,...,...,...,...,...,...,...,...,...
463375,1567274400,es.wikipedia,user,mobile-web,37,1,2019-08-31 18:00:00,2019,8,31,18
463376,1567260000,es.wikipedia,user,mobile-web,5108,1,2019-08-31 14:00:00,2019,8,31,14
463377,1567252800,es.wikipedia,user,desktop,8,1,2019-08-31 12:00:00,2019,8,31,12
463378,1567252800,de.wikipedia,user,mobile-app,76,1,2019-08-31 12:00:00,2019,8,31,12


In [13]:
def df_to_json(df):
    outputdir = "../../data/website/hourly"

    jsondic = {i:[] for i in range(24)}

    # for hour in range(24):
    #     # cumx = 0
    #     for _, (x, y) in df[df["hour"]==hour][["x", "y"]].iterrows():
    #         # cumx += x
    #         jsondic[hour].append({"x":int(cumx), "y":int(y)})

    for _, (hour, x, y) in df[["hour", "x", "y"]].iterrows():
        jsondic[hour].append({"x":int(x), "y":int(y)})

    tt = df["trafficType"].iloc[0]
    at = df["accessType"].iloc[0]
    domain = df["domain"].iloc[0]
    year = df["year"].iloc[0]
    month = df["month"].iloc[0]
    day = df["day"].iloc[0]

    outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
    outfiledir = "/".join(outfilepath.split("/")[:-1])

    if not os.path.exists(outfiledir):
        os.makedirs(outfiledir)

    with open(outfilepath, "w") as f:
        json.dump(jsondic, f)

In [14]:
df\
    .sort_values(["trafficType", "accessType", "domain", "year", "month", "day", "hour", "y"], ascending=[False, False, False, True, True, True, True, False])\
    .groupby(["trafficType", "accessType", "domain", "year", "month", "day"])\
    .apply(df_to_json)

""


# Monthly Data

In [15]:
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/sep2018.parquet" "../../data/website/databricks/monthly/sep2018.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/sep2019.parquet" "../../data/website/databricks/monthly/sep2019.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/oct2019.parquet" "../../data/website/databricks/monthly/oct2019.parquet"
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/aug2019.parquet" "../../data/website/databricks/monthly/aug2019.parquet"

In [9]:
df = pd.read_parquet("../../data/website/databricks/monthly/aug2019.parquet").drop_duplicates()
df["date"] = pd.to_datetime(df["timestamp"], unit="s")
df

,timestamp,domain,trafficType,accessType,sumcount,date
0,1566741600,zh.wikipedia,user,desktop,5158,2019-08-25 14:00:00
1,1567249200,zh.wikipedia,spider,mobile-web,972,2019-08-31 11:00:00
2,1566849600,en.wikipedia,user,mobile-web,73373,2019-08-26 20:00:00
3,1566824400,fr.wikipedia,user,mobile-app,11734,2019-08-26 13:00:00
4,1566961200,es.wikipedia,spider,desktop,5205,2019-08-28 03:00:00
...,...,...,...,...,...,...
22315,1564833600,zh.wikipedia,user,desktop,16255,2019-08-03 12:00:00
22316,1566190800,es.wikipedia,user,mobile-web,8188,2019-08-19 05:00:00
22317,1566705600,ru.wikipedia,spider,mobile-web,2613,2019-08-25 04:00:00
22318,1565496000,zh.wikipedia,spider,desktop,6516,2019-08-11 04:00:00


In [10]:
df.sort_values("date")

,timestamp,domain,trafficType,accessType,sumcount,date
7953,1564617600,es.wikipedia,spider,desktop,4541,2019-08-01 00:00:00
9856,1564617600,fr.wikipedia,user,mobile-web,4887,2019-08-01 00:00:00
13872,1564617600,zh.wikipedia,user,mobile-web,1577,2019-08-01 00:00:00
17850,1564617600,ru.wikipedia,user,desktop,4310,2019-08-01 00:00:00
15018,1564617600,es.wikipedia,spider,mobile-web,1228,2019-08-01 00:00:00
...,...,...,...,...,...,...
8789,1567292400,fr.wikipedia,user,mobile-web,7789,2019-08-31 23:00:00
18175,1567292400,ru.wikipedia,user,mobile-web,2137,2019-08-31 23:00:00
10907,1567292400,en.wikipedia,user,mobile-app,122581,2019-08-31 23:00:00
6699,1567292400,de.wikipedia,user,mobile-web,3532,2019-08-31 23:00:00


In [11]:
outputdir = "../../data/website/monthly"

problematicpaths = []

for tt in ["user", "spider"]:
    for at in ["desktop", "mobile-web", "mobile-app"]:
        for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
            for year in [2019]:
                for month in [8]:

                    jsondic = []

                    filtered_df = df[
                        (df["domain"] == domain) &
                        (df["accessType"] == at) &
                        (df["trafficType"] == tt) &
                        (df["date"].dt.year == year) &
                        (df["date"].dt.month == month)
                    ]

                    filtered_df.sort_values("timestamp", ascending=True, inplace=True)

                    for _, (date, y) in filtered_df[["date", "sumcount"]].iterrows():
                        x = f"{date.year:04d}-{date.month:02d}-{date.day:02d}-{date.hour:02d}"
                        jsondic.append({"x":x, "y":int(y)})

                    outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}.json"
                    outfiledir = "/".join(outfilepath.split("/")[:-1])  # the directory where the file is in

                    if not os.path.exists(outfiledir):
                        os.makedirs(outfiledir)

                    with open(outfilepath, "w") as f:
                        json.dump(jsondic, f)

In [76]:
# !rm ../../data/website/hourly/*/*/*/2019-10*.json
# !rm ../../data/website/monthly/*/*/*/2019-10.json